# Domain Term Review
本筆記提供快速檢視目前領域文本中常出現的詞彙，方便人工挑出應保留的專業詞與需排除的泛用詞。

執行順序：
1. 讀取 `config/domain_defaults.json` 中的預設路徑並擷取文字。
2. 依出現頻率列出候選詞，可調整排名數量或另外輸出為檔案。
3. 人工挑選專業詞後，可將結果貼回程式（例如建立白名單或黑名單）。

In [16]:
from pathlib import Path
import json, re, hashlib
from collections import Counter

ROOT = Path.cwd()
CANDIDATES = [ROOT] + list(ROOT.parents)
for base in CANDIDATES:
    candidate = base / 'config' / 'domain_defaults.json'
    if candidate.exists():
        CONFIG_PATH = candidate
        break
else:
    raise FileNotFoundError('找不到 config/domain_defaults.json，請確認工作目錄或專案結構。')

try:
    display_path = CONFIG_PATH.relative_to(ROOT)
except ValueError:
    display_path = CONFIG_PATH
print(f'使用設定檔: {display_path}')
with CONFIG_PATH.open('r', encoding='utf-8') as f:
    CONFIG = json.load(f)

allowed_suffixes = {s.lower() for s in CONFIG.get('allowed_suffixes', ['.txt', '.md', '.pdf'])}
ignore_keyword = CONFIG.get('ignore_keyword', '')
domain_paths = [Path(p).expanduser() for p in CONFIG.get('domain_paths', [])]

def iter_texts():
    seen_signatures = set()
    for base in domain_paths:
        if not base.exists():
            continue
        for path in base.rglob('*'):
            if not path.is_file():
                continue
            if ignore_keyword and ignore_keyword in path.as_posix():
                continue
            suffix = path.suffix.lower()
            if suffix not in allowed_suffixes:
                continue
            text = ''
            if suffix == '.pdf':
                try:
                    from pypdf import PdfReader
                except Exception as exc:
                    print(f'略過 {path} (缺少 pypdf 或解析失敗): {exc}')
                    continue
                try:
                    reader = PdfReader(str(path))
                    text = '\n'.join(page.extract_text() or '' for page in reader.pages)
                except Exception as exc:
                    print(f'略過 {path} (PDF 解析失敗): {exc}')
                    continue
            else:
                try:
                    text = path.read_text(encoding='utf-8', errors='ignore')
                except Exception as exc:
                    print(f'略過 {path} (讀取失敗): {exc}')
                    continue
            signature = (path.name.lower(), hashlib.md5(text.encode('utf-8', errors='ignore')).hexdigest())
            if signature in seen_signatures:
                continue
            seen_signatures.add(signature)
            weight = 4 if suffix == '.md' else 1
            yield text, weight

documents = list(iter_texts())
total_chars = sum(len(text) * weight for text, weight in documents)
print(f'收集到 {total_chars:,} 字元（加權後）。')

RE_CJK = re.compile(r'[\u4e00-\u9fff]{2,10}')
RE_EN = re.compile(r'[A-Za-z][A-Za-z0-9_\-]{2,}')
counts = Counter()
case_map = {}

for text, weight in documents:
    for m in RE_CJK.finditer(text):
        counts[m.group(0)] += weight
    for m in RE_EN.finditer(text):
        raw = m.group(0)
        key = raw.lower()
        if key not in case_map:
            case_map[key] = raw
        counts[key] += weight

def top_terms(n=200):
    items = []
    for token, freq in counts.items():
        shown = case_map.get(token, token)
        items.append((freq, len(shown), shown))
    items.sort(key=lambda tpl: (-tpl[0], -tpl[1], tpl[2]))
    return items[:n]

print('完成統計，可使用 `top_terms()` 查看結果。')


使用設定檔: /Users/iw/Documents/NTU/1141/whisper/whisper-rt-full/whisper-realtime/config/domain_defaults.json


Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)


收集到 5,299,172 字元（加權後）。
完成統計，可使用 `top_terms()` 查看結果。


In [17]:
top_terms()

[(4946, 2, '判決'),
 (4224, 2, '借名'),
 (3858, 2, '條第'),
 (3181, 2, '因此'),
 (1376, 4, '最高法院'),
 (1314, 3, '號民事'),
 (1263, 4, '台上字第'),
 (1236, 2, '登記'),
 (1231, 2, '例如'),
 (1184, 6, '年度台上字第'),
 (749, 4, '也就是說'),
 (715, 2, '然而'),
 (610, 2, '規定'),
 (546, 4, '條的規定'),
 (505, 4, '項的規定'),
 (489, 2, '原則'),
 (459, 2, '所以'),
 (457, 2, '關係'),
 (438, 3, '換言之'),
 (415, 3, '條規定'),
 (413, 2, '當然'),
 (406, 2, '項第'),
 (389, 4, '所得稅法'),
 (377, 3, '項規定'),
 (370, 3, '釋字第'),
 (357, 2, '刪除'),
 (356, 2, '問題'),
 (333, 2, '處分'),
 (332, 2, '義務'),
 (325, 2, '台上'),
 (321, 2, '使用'),
 (310, 7, '遺產及贈與稅法'),
 (284, 3, '條的第'),
 (273, 5, '稅捐稽徵法'),
 (272, 3, '接下來'),
 (264, 4, '簡單來說'),
 (260, 4, '特別公課'),
 (260, 4, '登記契約'),
 (252, 2, '行為'),
 (251, 4, '不在此限'),
 (251, 2, '動產'),
 (243, 3, '憲法第'),
 (242, 4, '借名登記'),
 (238, 2, '稅捐'),
 (233, 5, '租稅法總論'),
 (232, 2, '公司'),
 (232, 2, '此外'),
 (232, 2, '課程'),
 (223, 3, '的問題'),
 (219, 3, '民法第'),
 (214, 2, '日期'),
 (212, 2, '周次'),
 (212, 2, '節次'),
 (211, 2, '因為'),
 (211, 2, '認為'),
 (208, 4

In [18]:
TOP_N = 3000  # 可自行調整
terms = top_terms(TOP_N)
for rank, (freq, length, token) in enumerate(terms, start=1):
    print(f'{rank:>4}. {token}	{freq}')

   1. 判決	4946
   2. 借名	4224
   3. 條第	3858
   4. 因此	3181
   5. 最高法院	1376
   6. 號民事	1314
   7. 台上字第	1263
   8. 登記	1236
   9. 例如	1231
  10. 年度台上字第	1184
  11. 也就是說	749
  12. 然而	715
  13. 規定	610
  14. 條的規定	546
  15. 項的規定	505
  16. 原則	489
  17. 所以	459
  18. 關係	457
  19. 換言之	438
  20. 條規定	415
  21. 當然	413
  22. 項第	406
  23. 所得稅法	389
  24. 項規定	377
  25. 釋字第	370
  26. 刪除	357
  27. 問題	356
  28. 處分	333
  29. 義務	332
  30. 台上	325
  31. 使用	321
  32. 遺產及贈與稅法	310
  33. 條的第	284
  34. 稅捐稽徵法	273
  35. 接下來	272
  36. 簡單來說	264
  37. 特別公課	260
  38. 登記契約	260
  39. 行為	252
  40. 不在此限	251
  41. 動產	251
  42. 憲法第	243
  43. 借名登記	242
  44. 稅捐	238
  45. 租稅法總論	233
  46. 公司	232
  47. 此外	232
  48. 課程	232
  49. 的問題	223
  50. 民法第	219
  51. 日期	214
  52. 周次	212
  53. 節次	212
  54. 因為	211
  55. 認為	211
  56. 簡單來講	208
  57. 所謂	208
  58. 評析	205
  59. 原則上	203
  60. 在德國	200
  61. 方式	196
  62. 土地	192
  63. 所得	190
  64. 小結	189
  65. 評釋	188
  66. 土地稅法	185
  67. 課稅	184
  68. 法第	183
  69. 公課	181
  70. 法院	181
  71. 收入	179
  72. 換句話說	176

In [19]:
# 依據加權頻率挑選長度不超過 6 字元的候選專業詞彙，可調整參數
MAX_SPECIALIZED_LENGTH = 6
MAX_SPECIALIZED_RESULTS = 2000
SPECIALIZED_STOPWORDS = {
    '因此', '例如', '所以', '換言之', '也就是說', '此外', '問題', '方式', '行為', '因為', '然而',
    '就是', '結果', '結論', '討論', '其中', '可以', '以及', '如果', '原則', '還是', '然後',
    '例如說', '舉例', '舉例來說', '或是', '但是', '不過'
}

def extract_specialized_terms(max_length=MAX_SPECIALIZED_LENGTH, limit=MAX_SPECIALIZED_RESULTS, stopwords=None):
    ranked = []
    normalized_stopwords = {s.lower() for s in (stopwords or set())}
    for freq, length, token in top_terms(TOP_N):
        key = token.lower()
        if length > max_length:
            continue
        if key in normalized_stopwords:
            continue
        if key.isdigit():
            continue
        ranked.append((token, freq))
        if len(ranked) >= limit:
            break
    return ranked

specialized_terms = extract_specialized_terms(stopwords=SPECIALIZED_STOPWORDS)
print(f'候選專業詞彙 {len(specialized_terms)} 筆 (長度≤{MAX_SPECIALIZED_LENGTH})。')
for idx, (token, freq) in enumerate(specialized_terms, start=1):
    print(f'{idx:>3}. {token}	{freq}')


候選專業詞彙 2000 筆 (長度≤6)。
  1. 判決	4946
  2. 借名	4224
  3. 條第	3858
  4. 最高法院	1376
  5. 號民事	1314
  6. 台上字第	1263
  7. 登記	1236
  8. 年度台上字第	1184
  9. 規定	610
 10. 條的規定	546
 11. 項的規定	505
 12. 關係	457
 13. 條規定	415
 14. 當然	413
 15. 項第	406
 16. 所得稅法	389
 17. 項規定	377
 18. 釋字第	370
 19. 刪除	357
 20. 處分	333
 21. 義務	332
 22. 台上	325
 23. 使用	321
 24. 條的第	284
 25. 稅捐稽徵法	273
 26. 接下來	272
 27. 簡單來說	264
 28. 特別公課	260
 29. 登記契約	260
 30. 不在此限	251
 31. 動產	251
 32. 憲法第	243
 33. 借名登記	242
 34. 稅捐	238
 35. 租稅法總論	233
 36. 公司	232
 37. 課程	232
 38. 的問題	223
 39. 民法第	219
 40. 日期	214
 41. 周次	212
 42. 節次	212
 43. 認為	211
 44. 簡單來講	208
 45. 所謂	208
 46. 評析	205
 47. 原則上	203
 48. 在德國	200
 49. 土地	192
 50. 所得	190
 51. 小結	189
 52. 評釋	188
 53. 土地稅法	185
 54. 課稅	184
 55. 法第	183
 56. 公課	181
 57. 法院	181
 58. 收入	179
 59. 換句話說	176
 60. 土地增值稅	174
 61. 脫法	172
 62. 民事法裁判	170
 63. 號民事判決	170
 64. 款規定	170
 65. 年憲判字第	169
 66. 卷第	169
 67. 標準	169
 68. 裡面	168
 69. 比例原則	165
 70. 年的	164
 71. 釋字	161
 72. 實際上	160
 73. 所得稅	158
 74. 規範	158
 75. 首先	158
 76. 量

In [20]:
# 將候選專業詞彙輸出為 Markdown 檔案
from datetime import datetime

OUTPUT_MD_PATH = ROOT /  "domain_specialized_terms.md"
OUTPUT_MD_PATH.parent.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
lines = [
    "# 領域專業詞彙候選清單",
    "",
    f"- 產出時間: {timestamp}",
    f"- 使用設定檔: `{CONFIG_PATH}`",
    f"- 篩選條件: 長度≤{MAX_SPECIALIZED_LENGTH}, TOP_N={TOP_N}",
    "",
    "| 排名 | 詞彙 | 頻率 |",
    "| ---: | :--- | ---: |",
]

for idx, (token, freq) in enumerate(specialized_terms, start=1):
    safe_token = token.replace("|", "\\|")
    lines.append(f"| {idx} | {safe_token} | {freq} |")

OUTPUT_MD_PATH.write_text("\n".join(lines), encoding="utf-8")
print(f"已輸出 {len(specialized_terms)} 筆紀錄至 {OUTPUT_MD_PATH}")

已輸出 2000 筆紀錄至 /Users/iw/Documents/NTU/1141/whisper/whisper-rt-full/whisper-realtime/notebooks/domain_specialized_terms.md
